In [1]:
import os
import sys
import json

In [2]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## logging

In [3]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# you can set a tokenizer directly, or optionally let it default
# to the same tokenizer that was used previously for token counting
# NOTE: The tokenizer should be a function that takes in text and returns a list of tokens
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode,
    verbose=True  # set to true to see usage printed to the console
    )
callback_manager = CallbackManager([token_counter])

## create a new index from scratch
check [LlamaHub](https://llamahub.ai/) for different helpers to create an index.
export notion pages, etc. to markdown and put them into the data folder.
For now exclude the images, they currently blow up the chunk size limit.

In [4]:
# Load you data into 'Documents' a custom type by LlamaIndex
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('./data', recursive=True).load_data()

### add metadata

In [5]:
import re

# save control fields in the documents metadata
for doc in documents:tidy up

    # extract the Value of the control from the text with regex. controls are in the format: /nControl: Value/n
    contr = re.search(r'(?<=\nControl: ).*(?=\n)', doc.text)
    #check if a control value was found and write it to the metadata
    if contr:
        doc.metadata['Control'] = contr.group(0)
        print(doc.metadata['Control'])

NULL
NULL
LogWindow
AcceptAddSpace
NULL
AnalysisTabButton
NULL
AnnotationsTabButton
AnnotationsList
NULL
AutomaticCollabToggle
AvailableDataListView
ProjectAvailableOfflineToggle
AvailableSessionsList
NULL
BreadCrumbs
NULL
NULL
NULL
CollabMenu
StartCollab
StopCollab
NULL
ConnectedUsersList
CopyPublicLinkToSpaceButton
CopyPublicLinkToViewButton
NULL
NULL
NULL
NULL
DeleteExplorerButton
DuplicateSpaceButton
EditProjectData
NULL
NULL
ExpandLogButton
OpenExplorerButton
CloseExplorerButton
AddExplorerButton
ExplorerOptionsButton
NULL
NULL
NULL
NULL
NULL
NULL
ExplorerDataImporter
NULL
CloseInspectorButton
EditName
NULL
NULL
NULL
NULL
NULL
LoadProject
NULL
LocalDataTabButton; OpenLocalButton
NULL
LoginButton
NULL
MaxExtent
MinExtent
NULL
NULL
AddSessionButton
NULL
OfflineStartButtonControl
OpenInspectorButton
LiquidpediaButton
NULL
NULL
LoginPassword
NULL
NULL
NULL
NULL
NULL
RememberAccount
NULL
NULL
ProjectsTabButton; OpenProjectsButton
NULL
SessionNamingField
NULL
NULL
NULL
NULL
NULL
NULL
NU

In [6]:
# This is for when building the index from nodes instead Documents:
# Nodes are usefull when the same nodes should be part of several indexes
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [9]:
from llama_index import VectorStoreIndex
from llama_index import LangchainEmbedding, ServiceContext

service_context = ServiceContext.from_defaults(callback_manager=callback_manager) # use default settings but connect the token logger
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Embedding Token Usage: 628
Embedding Token Usage: 335
Embedding Token Usage: 329
Embedding Token Usage: 421
Embedding Token Usage: 1004
Embedding Token Usage: 545
Embedding Token Usage: 1510
Embedding Token Usage: 831
Embedding Token Usage: 279
Embedding Token Usage: 851
Embedding Token Usage: 706
Embedding Token Usage: 560
Embedding Token Usage: 757
Embedding Token Usage: 383
Embedding Token Usage: 425
Embedding Token Usage: 300
Embedding Token Usage: 565
Embedding Token Usage: 723
Embedding Token Usage: 285
Embedding Token Usage: 573
Embedding Token Usage: 499
Embedding Token Usage: 826
Embedding Token Usage: 345
Embedding Token Usage: 388
Embedding Token Usage: 1471
Embedding Token Usage: 1206
Embedding Token Usage: 9062
Embedding Token Usage: 412
Embedding Token Usage: 931
Embedding Token Usage: 1048
Embedding Token Usage: 2510
Embedding Token Usage: 1103
Embedding Token Usage: 4367
Embedding Token Usage: 3477


In [10]:
print(token_counter.total_embedding_token_count)

39655


## save the index

saves the index under /storage in json format

In [13]:
index.storage_context.persist()